## Mehmet Burak Erkan<br>
Osmanbey - IEEE Student<br>
Mastering Applied Data Science with Deep Learning, 2022 cohort<br>

<h1 align="center">NLP Project Final Project</h1>

# Introduction #
**Problem Statement**: Classifying Amazon reviews based on customer ratings using NLP

In [3]:
import warnings

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from textblob import TextBlob, Word
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,precision_score,recall_score,f1_score
from unicodedata import normalize

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Verimizi internetten çekip dosyamıza dahil ediyoruz.

In [4]:
import os
import pandas as pd

if os.path.isfile("Electronics_5.json"):
    df = pd.read_json("Electronics_5.json", lines=True)
else:
    df = pd.read_json(r"http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz", compression='gzip', lines=True)

display(df.head(10))

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011"
5,A2JXAZZI9PHK9Z,0594451647,"Billy G. Noland ""Bill Noland""","[3, 3]",I am using this with a Nook HD+. It works as d...,5,HDMI Nook adapter cable,1388707200,"01 3, 2014"
6,A2P5U7BDKKT7FW,0594451647,Christian,"[0, 0]",The cable is very wobbly and sometimes disconn...,2,Cheap proprietary scam,1398556800,"04 27, 2014"
7,AAZ084UMH8VZ2,0594451647,"D. L. Brown ""A Knower Of Good Things""","[0, 0]",This adaptor is real easy to setup and use rig...,5,A Perfdect Nook HD+ hook up,1399161600,"05 4, 2014"
8,AEZ3CR6BKIROJ,0594451647,Mark Dietter,"[0, 0]",This adapter easily connects my Nook HD 7&#34;...,4,A nice easy to use accessory.,1405036800,"07 11, 2014"
9,A3BY5KCNQZXV5U,0594451647,Matenai,"[3, 3]",This product really works great but I found th...,5,This works great but read the details...,1390176000,"01 20, 2014"


In [5]:
df.shape

(1689188, 9)

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1689188 entries, 0 to 1689187
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   reviewerID      1689188 non-null  object
 1   asin            1689188 non-null  object
 2   reviewerName    1664458 non-null  object
 3   helpful         1689188 non-null  object
 4   reviewText      1689188 non-null  object
 5   overall         1689188 non-null  int64 
 6   summary         1689188 non-null  object
 7   unixReviewTime  1689188 non-null  int64 
 8   reviewTime      1689188 non-null  object
dtypes: int64(2), object(7)
memory usage: 116.0+ MB
None


Datetime verisine dönüştürüyoruz.

In [7]:
from datetime import datetime

condition = lambda row: datetime.fromtimestamp(row).strftime("%m-%d-%Y")
df["unixReviewTime"] = df["unixReviewTime"].apply(condition)

"reviewTime" gereksiz olduğu için bu sütunu siliyoruz.

In [8]:
df.drop(labels="reviewTime", axis=1, inplace=True)

display(df.head())

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,06-02-2013
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,11-25-2010
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,09-09-2010
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",11-24-2010
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",09-29-2011


Üzerinde oynayacağımız veriler "reviewText" sütununda saklanıyor.

In [9]:
print(df["reviewText"].iloc[0])

We got this GPS for my husband who is an (OTR) over the road trucker.  Very Impressed with the shipping time, it arrived a few days earlier than expected...  within a week of use however it started freezing up... could of just been a glitch in that unit.  Worked great when it worked!  Will work great for the normal person as well but does have the "trucker" option. (the big truck routes - tells you when a scale is coming up ect...)  Love the bigger screen, the ease of use, the ease of putting addresses into memory.  Nothing really bad to say about the unit with the exception of it freezing which is probably one in a million and that's just my luck.  I contacted the seller and within minutes of my email I received a email back with instructions for an exchange! VERY impressed all the way around!


## NLP Pre-Processing ##


In [10]:
import html

df["reviewText"] = html.unescape(df["reviewText"])
df["reviewText"] = df["reviewText"].str.replace(pat= r"\&\#[0-9]+\;", repl="", regex=True).str.lower().replace("[^\w\s]","").str.replace("\d+","").str.replace("\n"," ").str.replace("\r","")

In [11]:
print(df["reviewText"].iloc[1689185])

im a big fan of the brainwavz s (actually all of their headphones  have yet to be disappointed with any of their products). the s has been my main set for active use (e.g., workouts, runs, etc.) since the flat cable is very durable and resistant to tangles. the s keeps all the good features of the s and adds to it  the sound quality is richer and better defined.thats not to say the s sounds poor  they are quite good, in fact. but the s are better. the highs are better defined and the midrange has more punch to it. the bass comes through clearly without moving into the harsh territory when the volume is pushed (as the ss can do). the overall sound quality is very pleasing.the build quality seems solid  as solid as the s or better. i love the flat cable! i know thats something that is not appreciated by everyone, but for me its been working out wonderfully. although this (as most other brainwavz headsets) comes with an excellent hard shell case, i usually tote my earbuds wrapped around m

In [ ]:
df["reviewText"] = df["reviewText"].apply(lambda x: word_tokenize(x)).apply(lambda x: [word for word in x if word not in list(stopwords.words("english"))]).apply(lambda x: WordNetLemmatizer().lemmatize(x)).apply(lambda x: " ".join(x))

### Aksanları Kaldırıyoruz ###


In [ ]:
df["reviewText"] = df["reviewText"].apply(lambda text: normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore"))

## Phrase Modeling ##
"phrasal" ifadelerin anlamlarını alabilmek için bu fonksiyonu kullanıyoruz

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

bi_gram = Phrases(df["reviewText"], min_count=300, threshold=50)

tri_gram = Phrases(bi_gram[df["reviewText"]], min_count=300, threshold=50)

In [ ]:
tokenized = [Phraser(tri_gram)[Phraser(bi_gram)[i]] for i in tokenized]

In [ ]:
tokenized = [list(filter(lambda x: len(x) > 1, document)) \
             for document in tokenized]


In [ ]:
%%time
tokenized_array = np.array(tokenized)

model_array = np.array([word_vec_df.loc[doc].mean(axis=0) for doc in tokenized_array])

Every document is provided the ground truth label by imposing its `overall` rating. This completes our finalized `model_df` dataframe.

In [ ]:
model_df = pd.DataFrame(model_array)
model_df["label"] = df["overall"]
y = model_df["label"] 
del model_df["label"]
display(model_df.head())

## Principal Component Analysis ##


In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=3)
x_r=pca.fit_transform(model_df)



In [ ]:
word_bank = ["nook", "phone", "tv", "good", "price"]

for word in word_bank[:]:
    related_vec = word_vec.wv.most_similar(word, topn=5)
    related_words = np.array(related_vec)[:,0]
    word_bank.extend(related_words)
    print(f"{word}: {related_words}")

## Named-Entity Recognition ###
Burada bazı kelimeleri genel sınıflarıyla etiketleyip metinleri sınıflandırıyoruz.

In [ ]:
helpful = df["helpful"].tolist()
most_helpful = max(helpful, key=lambda x: x[0])

most_helpful_idx = df["helpful"].astype(str) == str(most_helpful)
most_helpful_idx = df[most_helpful_idx].index

most_helpful_text = df["reviewText"].iloc[most_helpful_idx].values[0]

print(most_helpful_text)

We use `ner_dict`, a dictionary initialized as a list, to segregate the nouns in the `most_helpful_text` into the NER tags.

In [ ]:
%%time
import spacy

from collections import defaultdict

ner = spacy.load("en")

ner_helpful = ner(df["reviewText"].values)

ner_dict = defaultdict(list)
for entity in ner_helpful.ents:
    ner_dict[entity.label_].append(entity)

for NER, name in ner_dict.items():
    print(f"{NER}:\n{name}\n")

## Topic Modeling ##


In [ ]:
%%time
import multiprocessing

from gensim.models.ldamulticore import LdaMulticore

cores = multiprocessing.cpu_count()

num_topics = 10
bow_lda = LdaMulticore(bow, num_topics=num_topics, id2word=vocabulary, \
                       passes=5, workers=cores, random_state=42)

for token, frequency in bow_lda.show_topic(0, topn=5):
    print(token, frequency)

In [ ]:
for topic in range(0, num_topics):
    print(f"\nTopic {topic+1}:")
    for token, frequency in bow_lda.show_topic(topic, topn=5):
        print(f" {token}, {frequency}")

# Machine Learning #


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_r, y, stratify=y, test_size=0.25, random_state=42)

In [ ]:
def classification_(train,y):
    
    
    k = KNeighborsClassifier()
    svc = SVC()
    d = DecisionTreeClassifier()
    log = LogisticRegression()
    gbc = GradientBoostingClassifier()
    mn = MultinomialNB()
    rf = RandomForestClassifier()
    ab = AdaBoostClassifier()

    x = train
    y = y
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = .2, random_state=42)
    algos = [k,svc,d,log,gbc,mn,rf,ab]
    algos_name = ['KNeigbors','SVC','DecisionTree','LogisticRegr','GradientBoosting','Multinominal','RandomForest','AdaBoost']
    
    accuracy = []
    precision = []
    recall = []
    f1 = []
   
    result = pd.DataFrame(columns=['AccuracyScore','PrecisionScore','RecallScore','f1_Score'],index = algos_name)
    
    for i in algos:
        
        predict = i.fit(x_train,y_train).predict(x_test)
        
        accuracy.append(accuracy_score(y_test,predict))
        precision.append(precision_score(y_test,predict))
        recall.append(recall_score(y_test,predict))
        f1.append(f1_score(y_test,predict))
        

    
    
    result.AccuracyScore = accuracy
    result.PrecisionScore = precision
    result.RecallScore = recall
    result.f1_Score = f1
    
    
    return result.sort_values('f1_Score',ascending=False)

In [ ]:
classification_(x_r,y)

In [ ]:
import tensorflow as tf

x_train, x_test, y_train, y_test 
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation="relu", input_shape=(784,)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=128, epochs=40, verbose=1, validation_data=(x_test,y_test))

In [ ]:
_, accuracy = model.evaluate(test_images, test_labels)
print("Accuracy: %.2f" % (accuracy*100))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train","Test"], loc="upper left")
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train","Test"], loc="upper left")
plt.show()


At 53.3% on a perfectly balanced training data set, we have achieved a better result compared to the 20% accuracy of our baseline.

## Word Cloud ##
Using the true labels of the reviews, we can take the fifty most salient words in every rating and produce a word cloud. The same `stop_words` we derived from the NLTK library are excluded.

We see that some of the words are quite descriptive of the rating, with "problem" and "issue" frequently appearing in one-star reviews, and "quality" and "highly recommend" in top reviews.

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(stopwords = set(stop_words), min_font_size=10, \
                      max_font_size=50, max_words=50, \
                      background_color="white", colormap = "Oranges")

one_star_text = " ".join(df[df["overall"]==1]["reviewText"].values).lower()
two_star_text = " ".join(df[df["overall"]==2]["reviewText"].values).lower()
three_star_text = " ".join(df[df["overall"]==3]["reviewText"].values).lower()
four_star_text = " ".join(df[df["overall"]==4]["reviewText"].values).lower()
five_star_text = " ".join(df[df["overall"]==5]["reviewText"].values).lower()

text_list = [one_star_text, two_star_text, three_star_text, \
             four_star_text, five_star_text]

for index, text in enumerate(text_list):
    f, axes = plt.subplots(figsize=(10,7))
    wordcloud.generate(text)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title(f"Word Cloud for {index+1}-Star Ratings")
    plt.axis("off")
    plt.show()